In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
file_path = '/content/hdx_hapi_food_price_lka.csv'
df = pd.read_csv(file_path)
df.head()

,location_code,has_hrp,in_gho,provider_admin1_name,provider_admin2_name,admin1_code,admin1_name,admin2_code,admin2_name,admin_level,...,lon,commodity_category,commodity_name,unit,price_flag,price_type,currency_code,price,reference_period_start,reference_period_end
0,#country+code,#meta+has_hrp,#meta+in_gho,#adm1+name+provider,#adm2+name+provider,#adm1+code,#adm1+name,#adm2+code,#adm2+name,#adm+level,...,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#date+start,#date+end
1,LKA,False,False,Central,Kandy,LK2,Central,LK21,Kandy,2,...,80.59,cereals and tubers,Rice (medium grain),KG,actual,Retail,LKR,247.5,2025-01-15,2025-02-14
2,LKA,False,False,Central,Kandy,LK2,Central,LK21,Kandy,2,...,80.59,"meat, fish and eggs",Eggs,Unit,actual,Retail,LKR,33.85,2025-01-15,2025-02-14
3,LKA,False,False,Central,Kandy,LK2,Central,LK21,Kandy,2,...,80.59,"meat, fish and eggs","Fish (dry, katta)",KG,actual,Retail,LKR,2057.5,2025-01-15,2025-02-14
4,LKA,False,False,Central,Kandy,LK2,Central,LK21,Kandy,2,...,80.59,"meat, fish and eggs",Fish (goldstripe sardinella),KG,actual,Retail,LKR,980,2025-01-15,2025-02-14


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20935 entries, 0 to 20934
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   location_code           20935 non-null  object
 1   has_hrp                 20935 non-null  object
 2   in_gho                  20935 non-null  object
 3   provider_admin1_name    20789 non-null  object
 4   provider_admin2_name    20789 non-null  object
 5   admin1_code             20789 non-null  object
 6   admin1_name             20789 non-null  object
 7   admin2_code             20789 non-null  object
 8   admin2_name             20789 non-null  object
 9   admin_level             20935 non-null  object
 10  market_name             20935 non-null  object
 11  lat                     20789 non-null  object
 12  lon                     20789 non-null  object
 13  commodity_category      20935 non-null  object
 14  commodity_name          20935 non-null  object
 15  un

## Handling Duplicates

In [ ]:
# Show duplicate rows
duplicates = df[df.duplicated()]
print("Duplicate rows:",duplicates)

Duplicate rows: Empty DataFrame
Columns: [location_code, has_hrp, in_gho, provider_admin1_name, provider_admin2_name, admin1_code, admin1_name, admin2_code, admin2_name, admin_level, market_name, lat, lon, commodity_category, commodity_name, unit, price_flag, price_type, currency_code, price, reference_period_start, reference_period_end]
Index: []

[0 rows x 22 columns]


## Handling Missing Values

In [ ]:
# Step 1: Fill missing values using forward fill
df = df.fillna(method='ffill')

# Step 2: Fill any remaining missing values using backward fill
df = df.fillna(method='bfill')


<ipython-input-4-4b845ed9c9cd>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
<ipython-input-4-4b845ed9c9cd>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


## Addtional Pre processing

In [ ]:
# Removing unecessary columns
df.drop(['location_code', 'has_hrp', 'in_gho', 'admin1_code', 'admin2_code', 'admin_level', 'price_flag', 'currency_code'], axis=1, inplace=True)


In [ ]:
# Renaming Columns
df = df.rename(columns={
'provider_admin1_name': 'Provider_Admin1_Name',
'provider_admin2_name': 'Provider_Admin2_Name',
'admin1_name': 'Admin1_Name',
'admin2_name': 'Admin2_Name',
'market_name': 'Market_Name',
'lat': 'Latitude',
'lon': 'Longitude',
'commodity_category': 'Commodity_Category',
'commodity_name': 'Commodity_Name',
'unit': 'Unit',
'price_type': 'Price_Type',
'price': 'Price',
'reference_period_start': 'Reference_Period_Start',
'reference_period_end': 'Reference_Period_End'
})

In [ ]:
# Ensure 'Price' is numeric and drop rows where it's missing
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df = df[df['Price'].notna()]

# Drop rows with missing start or end dates
df = df[df['Reference_Period_Start'].notna() & df['Reference_Period_End'].notna()]

# Remove rows with any missing or invalid ('nun') values
df_cleaned = df.dropna()
df_cleaned = df_cleaned[~df_cleaned.astype(str).apply(lambda x: x.str.contains('nun')).any(axis=1)]


In [ ]:
# Standardize units
df['Standardized_Price'] = df.apply(
    lambda row: row['Price'] / 1000 if 'ML' in row['Unit']
    else row['Price'] if 'KG' in row['Unit']
    else row['Price'] if 'Unit' in row['Unit']
    else row['Price'],
    axis=1
)


In [ ]:
# Convert date columns to datetime and extract months
date_cols = ['Reference_Period_Start', 'Reference_Period_End']
df[date_cols] = df[date_cols].apply(pd.to_datetime)

df['Start_Month'], df['End_Month'] = df['Reference_Period_Start'].dt.month, df['Reference_Period_End'].dt.month


In [ ]:
# Group by month and commodity, then calculate the mean of 'Standardized_Price'
df_monthly = df.groupby(['Start_Month', 'Commodity_Name'])['Standardized_Price'].mean().reset_index()

# Rename the 'Standardized_Price' column to 'Monthly_Average_Price'
df_monthly.rename(columns={'Standardized_Price': 'Monthly_Average_Price'}, inplace=True)


In [ ]:
# Filtering outliers
mean_price = df['Standardized_Price'].mean()
std_price = df['Standardized_Price'].std()

upper_limit = mean_price + 3 * std_price
lower_limit = mean_price - 3 * std_price

df = df[(df['Standardized_Price'] < upper_limit) & (df['Standardized_Price'] > lower_limit)]


In [ ]:
# Add descriptive statistics
# Step 1: Calculate the mean of 'Standardized_Price' for each 'Commodity_Name'
mean_price = df.groupby('Commodity_Name')['Standardized_Price'].mean()
df['Price_Mean'] = df['Commodity_Name'].map(mean_price)

median_price = df.groupby('Commodity_Name')['Standardized_Price'].median()
df['Price_Median'] = df['Commodity_Name'].map(median_price)

std_price = df.groupby('Commodity_Name')['Standardized_Price'].std()
df['Price_Std'] = df['Commodity_Name'].map(std_price)


<ipython-input-12-5d03d32e298a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price_Mean'] = df['Commodity_Name'].map(mean_price)
<ipython-input-12-5d03d32e298a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price_Median'] = df['Commodity_Name'].map(median_price)
<ipython-input-12-5d03d32e298a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [ ]:
# Step 1: Define the commodity mapping dictionary
commodity_mapping = {
    'Bananas': 'Vegetables and Fruits',
    'Carrots': 'Vegetables and Fruits',
    'Coconut': 'Vegetables and Fruits',
    'Eggplants': 'Vegetables and Fruits',
    'Onions (imported)': 'Vegetables and Fruits',
    'Onions (red, local)': 'Vegetables and Fruits',
    'Papaya': 'Vegetables and Fruits',
    'Pineapples': 'Vegetables and Fruits',
    'Pumpkin': 'Vegetables and Fruits',
    'Snake gourd': 'Vegetables and Fruits',
    'Tomatoes': 'Vegetables and Fruits',
    'Potatoes (imported)': 'Cereals and Tubers',
    'Potatoes (local)': 'Cereals and Tubers',
    'Rice (medium grain)': 'Cereals and Tubers',
    'Rice (white)': 'Cereals and Tubers',
    'Eggs': 'Meat, Fish and Eggs',
    'Fish (dry, sprats)': 'Meat, Fish and Eggs',
    'Fish (goldstripe sardinella)': 'Meat, Fish and Eggs',
    'Fish (sail fish)': 'Meat, Fish and Eggs',
    'Fish (skipjack tuna)': 'Meat, Fish and Eggs',
    'Fish (trenched sardinella)': 'Meat, Fish and Eggs',
    'Fish (yellowfin tuna)': 'Meat, Fish and Eggs',
    'Fish (jack)': 'Meat, Fish and Eggs',
    'Meat (chicken, broiler)': 'Meat, Fish and Eggs',
    'Meat (chicken, fresh)': 'Meat, Fish and Eggs',
    'Beans': 'Pulses and Nuts',
    'Beans (mung)': 'Pulses and Nuts',
    'Cowpeas (whole, average)': 'Pulses and Nuts',
    'Lentils': 'Pulses and Nuts',
    'Oil (coconut)': 'Oil and Fats',
    'Chili (red, dry raw)': 'Miscellaneous Food'
}

# Map 'Commodity_Name' to 'Commodity_Category' based on the dictionary
df['Commodity_Category'] = df['Commodity_Name'].map(commodity_mapping)
# Fill any missing values in 'Commodity_Category' with existing values
df['Commodity_Category'] = df['Commodity_Category'].fillna(df['Commodity_Category'])


<ipython-input-13-3c53a8f1a4bf>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commodity_Category'] = df['Commodity_Name'].map(commodity_mapping)
<ipython-input-13-3c53a8f1a4bf>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commodity_Category'] = df['Commodity_Category'].fillna(df['Commodity_Category'])


In [ ]:
df.head()

,Provider_Admin1_Name,Provider_Admin2_Name,Admin1_Name,Admin2_Name,Market_Name,Latitude,Longitude,Commodity_Category,Commodity_Name,Unit,Price_Type,Price,Reference_Period_Start,Reference_Period_End,Standardized_Price,Start_Month,End_Month,Price_Mean,Price_Median,Price_Std
1,Central,Kandy,Central,Kandy,Kandy,7.29,80.59,Cereals and Tubers,Rice (medium grain),KG,Retail,247.50,2025-01-15,2025-02-14,247.50,1,2,218.205444,220.00,26.085555
2,Central,Kandy,Central,Kandy,Kandy,7.29,80.59,"Meat, Fish and Eggs",Eggs,Unit,Retail,33.85,2025-01-15,2025-02-14,33.85,1,2,44.801405,46.69,9.244566
3,Central,Kandy,Central,Kandy,Kandy,7.29,80.59,NaN,"Fish (dry, katta)",KG,Retail,2057.50,2025-01-15,2025-02-14,2057.50,1,2,2059.112421,2080.99,265.540600
4,Central,Kandy,Central,Kandy,Kandy,7.29,80.59,"Meat, Fish and Eggs",Fish (goldstripe sardinella),KG,Retail,980.00,2025-01-15,2025-02-14,980.00,1,2,870.519979,885.00,238.003939
5,Central,Kandy,Central,Kandy,Kandy,7.29,80.59,"Meat, Fish and Eggs",Fish (skipjack tuna),KG,Retail,1163.00,2025-01-15,2025-02-14,1163.00,1,2,1150.981957,1147.50,315.694520


In [ ]:
# Save DataFrame to CSV file
df.to_csv('Final_Food_Prices.csv', index=False)
